# Funny birds to Time Series

In [ ]:
import os
import glob
import cv2
import json
import enum

from tqdm.auto import tqdm
from PIL import Image
import numpy as np

The enumerate `Center` indicates which center to use to obtain the time series:

- `Center.IMG`. Center of the image.
- `Center.CONTOUR`. The center of each contour. The resulting time series will vary a lot.
- `Center.FIRST_CONTOUR`.

In [ ]:

class Center(enum.Enum):
    IMG = 0
    CONTOUR = 1
    FIRST_CONTOUR = 2

    def __str__(self):
        claus = {0: "I", 1: "C", 2: "CO"}

        return claus[self.value]


In [ ]:
TEST = True

BODY_COLOR_PARTS = 170
BACKGROUND_COLOR_PARTS = 255
CENTER = Center.FIRST_CONTOUR

BACKGROUND_COLOR_IMG = (85, 85, 153)

## Mask creation

In [ ]:
def are_all_similar(a, b, c, tolerance=5):
    """ Determine if all three arrays are similar within a given tolerance.

    Args:
        a: Numpy array of values.
        b: Numpy array of values.
        c: Numpy array of values.
        tolerance: Tolerance level for similarity.

    Returns:
        Boolean numpy array indicating if all three arrays are similar within the tolerance.
    """
    return (
        np.logical_and(abs(a - b) <= tolerance,
                       np.logical_and(abs(a - c) <= tolerance, abs(b - c) <= tolerance))
    )


def get_mask(parts, tolerance=5):
    """ Returns a binary mask from a parts image.

    A parts image indicated each pixel corresponding to a different part. The background are always 

    Args:
        parts:  Image array of shape (H, W, 3), where each pixel's RGB value indicates the part label.
        tolerance: Tolerance level for color similarity.

    Returns:
        Numpy array of shape (H, W) representing the binary mask, where 1 indicates foreground and 0 indicates background.
    """
    parts_unic = parts.reshape(-1, parts.shape[2])
    parts_unic = np.unique(parts_unic, axis=0)
    parts_unic = parts_unic.astype(np.float64)
    diff = are_all_similar(parts_unic[:, 0], parts_unic[:, 1], parts_unic[:, 2], tolerance)

    values = parts_unic[diff]
    mask = np.zeros((256, 256), dtype=np.uint8)
    for val in values:
        if (val[0] == BODY_COLOR_PARTS) or (val[0] == BACKGROUND_COLOR_PARTS):
            continue
        # Binary mask of the object, adding each part
        mask = mask + np.logical_and.reduce(
            (parts[:, :, 0] == val[0], parts[:, :, 1] == val[1], parts[:, :, 2] == val[2]))

    return 1 - mask


def get_time_series(mask, center=None, return_points=False):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = sorted(cnts, key=lambda x: x.size, reverse=True)

    multiple_cnts = (len(cnts) > 1) and (cnts[1].size > 500)

    cnt = cnts[0]

    if center is None:
        M = cv2.moments(cnt)
        center = int(M["m01"] / M["m00"]), int(M["m10"] / M["m00"])

    time_series = []

    for point in cnt:
        point = point[0]
        time_series.append(
            float(np.sqrt((point[0] - center[0]) ** 2 + (point[1] - center[1]) ** 2)))

    if return_points:
        return time_series, center, multiple_cnts, cnt
    else:
        return time_series, center, multiple_cnts

In [ ]:
if TEST:
    part_map_paths = "./data/funny_birds/v2/test_part_map"
    inter_map_paths = "./data/funny_birds/v2/test_interventions"
else:
    part_map_paths = "./data/funny_birds/v2/train_part_map"
    inter_map_paths = None

IMG_PATH = f"./{part_map_paths}/**/*.png"

In [ ]:
images_paths = sorted(glob.glob(IMG_PATH))

classes = set()
for p in images_paths:
    classe = int(p.split(os.path.sep)[-2])

    if classe not in classes:
        classes.add(classe)

cls_dict = {}
for i, classe in enumerate(classes):
    cls_dict[classe] = i

In [ ]:
if TEST:
    folder_name = "test"
else:
    folder_name = "train"

for i, img_path in enumerate(tqdm(images_paths)):
    path_parts = img_path.split(os.path.sep)
    bird_class = path_parts[-2]
    bird_class_translated = cls_dict[int(bird_class)]

    file_img_name = path_parts[-1]
    img_name = file_img_name.split(".")[0]

    img_parts = Image.open(img_path)
    img_parts = np.array(img_parts)[:, :, :-1]
    mask = get_mask(img_parts, 40)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))
    mask[mask != 0] = 1

    if mask.max() == 0:
        print(f"Image {img_path} empty")
        continue

    # Path of the mask
    out_path = os.path.join("output", f"{folder_name}_{CENTER}",
                            str(bird_class_translated).zfill(3), img_name)
    os.makedirs(out_path, exist_ok=True)
    cv2.imwrite(f"./{out_path}/image.png", mask * 255)

    # To Time series
    center_img = mask.shape[0] // 2, mask.shape[1] // 2

    center = None

    time_series, center_cnt, multiple_cnts, cnt = get_time_series(mask.astype(np.uint8), return_points=True)

    if multiple_cnts:
        print(f"Image {str(i).zfill(5)} multiple cnts")
        continue

    if CENTER is Center.IMG:
        center = center_img
    elif CENTER is Center.CONTOUR:  # Center of the first contour
        center = center_cnt

    with open(f"./{out_path}/time_series.json", "w") as final:
        json.dump(time_series, final)

    # Saving interventions
    if inter_map_paths is not None:
        for interv_paths in tqdm(
                sorted(glob.glob(f"./{inter_map_paths}/{bird_class}/{img_name}/*.png")),
                leave=False):
            name_intervent = interv_paths.split(os.path.sep)[-1].split(".")[0]

            intervention = np.array(Image.open(interv_paths))[:, :, :-1]

            logical_mask = 1 - np.logical_and.reduce(
                (intervention[:, :, 0] == BACKGROUND_COLOR_IMG[0],
                 intervention[:, :, 1] == BACKGROUND_COLOR_IMG[1],
                 intervention[:, :, 2] == BACKGROUND_COLOR_IMG[2])
            )
            logical_mask = mask * logical_mask

            time_series, _, _ = get_time_series(logical_mask.astype(np.uint8), center)

            with open(f"./{out_path}/{name_intervent}.json", "w") as final:
                json.dump(time_series, final)

            cv2.imwrite(f"./{out_path}/{name_intervent}.png", logical_mask * 255)